In [28]:
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [29]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.com/Best-Sellers-Beauty/zgbs/beauty/ref=zg_bs_pg_'+str(pageNo)+'?_encoding=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    content = r.content
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price_range = d.find('span', attrs={'class':'a-size-base a-color-price'})
        rank = d.find('span', attrs={'class':'zg-badge-text'})

        all1=[]

        if name is not None:
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if rating is not None:
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price_range is not None:
            all1.append(price_range.text)
        else:
            all1.append('0')
            
        if rank is not None:
            all1.append(rank.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Product_Name','Rating','Customers_Rated', 'Price_Range', 'Rank'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')
